<a href="https://colab.research.google.com/github/linyuehzzz/hedetniemi_distance/blob/master/all_pair_distance_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**CUDA for all-pair distance algorithms**
CUDA parallelism for all-pair distance algorithms.  
Yue Lin (lin.3326 at osu.edu)  
Created: 6/12/2020

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#### **Install packages** 

In [2]:
!pip install timeout-decorator

  Created wheel for timeout-decorator: filename=timeout_decorator-0.4.1-cp36-none-any.whl size=5021 sha256=bd334f97cbb4413964ca7123527430a67413c97c789e6a7778f3cf708456b1d7
  Stored in directory: /root/.cache/pip/wheels/f1/e6/ea/7387e3629cb46ba65140141f972745b823f4486c6fe884ccb8
Successfully built timeout-decorator


#### **CUDA device query** 

In [1]:
!nvcc --version
from numba import cuda
print(cuda.gpus)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
<Managed Device 0>


In [7]:
%cd /usr/local/cuda-10.1/samples/1_Utilities/deviceQuery
!ls

/usr/local/cuda-10.1/samples/1_Utilities/deviceQuery
deviceQuery.cpp  Makefile  NsightEclipse.xml  readme.txt


In [9]:
!make
!./deviceQuery

make: Nothing to be done for 'all'.
./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla P100-PCIE-16GB"
  CUDA Driver Version / Runtime Version          10.1 / 10.1
  CUDA Capability Major/Minor version number:    6.0
  Total amount of global memory:                 16281 MBytes (17071734784 bytes)
  (56) Multiprocessors, ( 64) CUDA Cores/MP:     3584 CUDA Cores
  GPU Max Clock rate:                            1329 MHz (1.33 GHz)
  Memory Clock rate:                             715 Mhz
  Memory Bus Width:                              4096-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory: 

#### **Read graph data** 

In [3]:
%cd '/content/gdrive/My Drive/Colab Notebooks/hedetniemi_matrix_sum'

## Number of nodes (100/1,000/10,000/100,000/1,000,000)
nodes = 100
print('Nodes: ', nodes)
## Total degree
degree = 3
print('Degree: ', degree)

data = []
with open('graph_n' + str(nodes) + '_d' + str(degree) + '.txt', 'r') as f:
  lines = f.read().splitlines()
  for line in lines:
    l = line.split()
    item = [int(l[0]), int(l[1]), float(l[2])]
    data.append(item)

print(data[0])

/content/gdrive/My Drive/Colab Notebooks/hedetniemi_matrix_sum
Nodes:  100
Degree:  3
[77, 86, 89.39726376738572]


#### **Configure CUDA** 

In [4]:
NUM_THREADS = 32

def get_cuda_execution_config(m, n):
    dimBlock = (NUM_THREADS, NUM_THREADS)
    dimGrid = ((n // dimBlock[0]) + 1, (m // dimBlock[1]) + 1)
    return dimGrid, dimBlock

dimGrid, dimBlock = get_cuda_execution_config(nodes, nodes)
print('dimGrid: ', dimGrid)
print('dimBlock: ', dimBlock)

dimGrid:  (4, 4)
dimBlock:  (32, 32)


#### **Hedetniemi distance** 

##### Construct distance matrix

In [8]:
from timeit import default_timer
from numba import *
import numpy as np

@njit
def distance_matrix(graph):
  ## calculate number of nodes
  n = int(np.amax(graph[:,1]))

  ## calculate distance matrix
  dist_mtx = np.full((n,n), np.inf)
  for g in prange(graph.shape[0]):
    i = int(graph[g][0]) - 1
    j = int(graph[g][1]) - 1
    d = graph[g][2]
    dist_mtx[i,j] = d
    dist_mtx[j,i] = d

  ## set diagonal to 0
  np.fill_diagonal(dist_mtx, 0)
 
  return dist_mtx, n


## print time costs
try:
  start = default_timer()
  dist_mtx, n = distance_matrix(np.array(data)) 
  stop = default_timer()
  print('Time: ', stop - start)
except:
  print('Time: inf')
  raise

Time:  0.2530858050004099


##### Calculate Hedetniemi Matrix Sum

In [17]:
from timeit import default_timer
from numba import cuda, njit
from operator import *
import numpy as np

@njit
def init_mtx(n):
  return np.full((n,n), np.inf)

@cuda.jit
def cal_mtx(mtx_a_t_1, matrix, mtx_a_t, n):
  row = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
  col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x

  if row < n and col < n:
    a = mtx_a_t_1[row]
    b = matrix[:,col]

    summ = np.inf
    for k in range(n):
      summ = min(summ, a[k] + b[k])
    mtx_a_t[row,col] = summ

@njit
def cmp_mtx(mtx_a_t, mtx_a_t_1, p=True):
  if np.array_equal(mtx_a_t, mtx_a_t_1):
    p =  False
  else:
    mtx_a_t_1 = mtx_a_t

  return p

def hede_distance(matrix, n):
  mtx_a_t = init_mtx(n)
  mtx_a_t_1 = matrix

  p = True
  while p:
    cal_mtx[dimGrid, dimBlock](mtx_a_t_1, matrix, mtx_a_t, n)
    cuda.synchronize()    
    p = cmp_mtx(mtx_a_t, mtx_a_t_1, p)
  
  return mtx_a_t


## print time costs
try:
  start = default_timer()
  mtx_a_t = hede_distance(dist_mtx, n)
  stop = default_timer()
  print('Time: ', stop - start)
except:
  print('Time: inf')
  raise

## print shortest path matrix
with open('hedet_mtx_nb_cuda.txt', 'w') as fw:
  fw.write('\n'.join(['\t'.join([str(cell) for cell in row]) for row in mtx_a_t.tolist()]))

Time: inf


CudaAPIError: ignored

#### **Compare results** 

In [0]:
!diff 'hedet_mtx_list.txt' 'hedet_mtx_nb_cuda.txt'